In [1]:
%use s2, algoquant, krangl, lets-plot

// define a strategy
class SMA2Strategy(val product: Product, val fastLag: Int, val slowLag: Int, val scale: Double = 1.0) : SimpleOrderBookUpdateStrategy() {

    private val signal = MovingAverageCrossover(fastLag, slowLag)

    override fun computeOrders(time: LocalDateTime, orderBook: OrderBook, blotter: TradeBlotter): Collection<Order> {
        val crossovers = updateSignal(time, orderBook.mid())
        val position = blotter.position(product)
        var qty = 0.0
        for (crossover in crossovers) { // zero or one crossover
            if (crossover.state == Crossover.State.ABOVE && position < scale) { // cross above
                qty = scale - position
            } else if (crossover.state == Crossover.State.BELOW && position > -scale) { // cross below
                qty = -scale - position
            }
        }
        return if (DoubleUtils.isZero(qty, 1e-5)) emptyList() // no order
        else listOf(OrderUtils.newMarketOrder(product, qty))
    }

    private fun updateSignal(time: LocalDateTime, price: Double): List<Crossover.Signal> {
        signal.update(time, price)
        return signal.value()
    }
}

val simulatedInterval: LocalDateTimeInterval = LocalDateTimeInterval.of(
    Period.ofYears(5),
    LocalDate.of(2020, 1, 1).atStartOfDay()
)
val strategy = SMA2Strategy(
    product = USDStock.newInstance("AAPL", Exchange.NASDAQ),
    fastLag = 1,
    slowLag = 2
)

// backtest the strategy
print("running simulation...")
val report = SimpleSimulatorBuilder()
    .withInitialCash(100.0)
    .withOrderBookUpdates(
        OrderBookCaches(
            strategy.product,
            NMOrderBookCacheFactory().newInstance(
                strategy.product as Stock,
                simulatedInterval
            )
        )
    )
    .withMeasures(RiskMeasures().measures(simulatedInterval, 0.01))
    .build()
    .run(strategy, simulatedInterval)
println("done")

// display measures in a table
val values: MutableList<Any> = ArrayList()
report.measureKeys().forEach{
    values.add(it)
    values.add(report.getPerformance(it))
}
val df = dataFrameOf("Measure", "Value")(values)

df.print(title="Performance Measures", maxRows=df.nrow, maxDigits=4, rowNumbers=false)

// plot NAV curve
val date = report.navCache().map { it.time().toInstant(ZoneOffset.UTC) }.toList()
val nav = report.navCache().map { it.data() }.toList()
val navHistory = mapOf("Date" to date, "NAV" to nav)

var p = lets_plot(navHistory)
p += geom_line(color="dark_green") {x="Date"; y="NAV"} + scale_x_datetime()
p + ggsize(800, 450)

running simulation...

java.lang.RuntimeException: java.lang.reflect.InvocationTargetException
dev.nm.s2.data.DataCommand.void(fb:255)
dev.nm.s2.data.DataCommand.table(fb:117)
dev.nm.s2.app.algoquant.data.NMOrderBookCacheFactory.newInstance(NMOrderBookCacheFactory.java:50)
Line_14_jupyter.<init>(Line_14.jupyter.kts:46)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:96)
kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:41)
kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEva